In [1]:
# original way

# Purpose filter hcmi data for all features found in top gexp TMP model
# if a feature is not found in hcmi then that's ok, we will ignore

import pandas as pd
import numpy as np
import statistics
import scipy.stats as stats
import json
#######
cancer = 'PAAD'
run = 'HCMITumorModel'
#######

In [2]:
def df_read(CANCER, SAMPLE_TYPE):
    '''
    read in appropriate file and filter for TMP 
    important genes. provide HCMI abbrev for cancer cohort
    and sample type (Model or Tumor).
    outputs filtered df
    '''
    # if needed, merge the two matrices 
    if CANCER == 'LUNG' or CANCER == 'ESO':
        if CANCER == 'LUNG':
            file = '../data/midway.freeze.v2/{}_GEXP/{}_GEXP_prep2_{}.tsv'.format(CANCER, 'LUAD', SAMPLE_TYPE)
            df = pd.read_csv(file, sep='\t', index_col=0).sort_index(axis=0)
            file = '../data/midway.freeze.v2/{}_GEXP/{}_GEXP_prep2_{}.tsv'.format(CANCER, 'LUSC', SAMPLE_TYPE)
            df2 = pd.read_csv(file, sep='\t', index_col=0).sort_index(axis=0)
            assert list(df.index)==list(df2.index) # req for df concat using
        elif CANCER == 'ESO':
            file = '../data/midway.freeze.v2/{}_GEXP/{}_GEXP_prep2_{}.tsv'.format(CANCER, 'ESCC', SAMPLE_TYPE)
            df = pd.read_csv(file, sep='\t', index_col=0).sort_index(axis=0)
            file = '../data/midway.freeze.v2/{}_GEXP/{}_GEXP_prep2_{}.tsv'.format(CANCER, 'GEA', SAMPLE_TYPE)
            df2 = pd.read_csv(file, sep='\t', index_col=0).sort_index(axis=0)
            assert list(df.index)==list(df2.index) # req for df concat using        

        # Add any new fts from second CANCER cohort that isn't in first cohort
        b= list(df2.columns)
        a= list(df.columns)
        new = [ft for ft in b if ft not in a]
        if len(new) !=0:
            s1 = df2[new]
            df =pd.concat([df, s1], join="outer", axis = 1)
    else:
        file = '../data/midway.freeze.v2/{}_GEXP/{}_GEXP_prep2_{}.tsv'.format(CANCER, CANCER, SAMPLE_TYPE)
        df = pd.read_csv(file, sep='\t', index_col=0)
        if CANCER == 'LGGGBM':
            if 'HCM-BROD-1124-C16-06A' in list(df.index):
                print('applying manual removal for HCM-BROD-1124-C16-06A in LGGGBM')
                df = df.drop(['HCM-BROD-1124-C16-06A' ], axis=0, )
            if 'HCM-BROD-1124-C16-85B' in list(df.index):
                print('applying manual removal for HCM-BROD-1124-C16-85B in LGGGBM')
                df = df.drop(['HCM-BROD-1124-C16-85B' ], axis=0, )
    return df


def intersection(lst1, lst2):
    lst3 = [v for v in lst1 if v in lst2]
    return lst3


# def normalize_data(DATA, WHERE_GENE):
#     '''input DATA dataframe and if WHERE_GENE are found in "row" or "col" 
#     normalizes by divide each gexp_value by sum of all gexp vales then *1,000
#     '''
#     results = {}
#     # Normalize across WHERE_GENE for each sample
#     if WHERE_GENE == 'col':
#         for sample in DATA.index:
#             gexp = DATA.loc[sample,]
#             # divide each gexp_value by sum of all gexp vales then *1,000
#             total_sum = sum(gexp)
#             normalized_gexp = [(g/total_sum)*1000 for g in gexp]
#             # save
#             assert sample not in results, 'repeated sample names. can not have in python dictionary'
#             results[sample]=normalized_gexp
#             normalized_matrix = pd.DataFrame.from_dict(results).transpose()
#             normalized_matrix.columns = DATA.columns
#     elif WHERE_GENE == 'row':
#         for sample in DATA.columns:
#             gexp = DATA[sample]
#             # divide each gexp_value by sum of all gexp vales then *1,000
#             total_sum = sum(gexp)
#             normalized_gexp = [(g/total_sum)*1000 for g in gexp]
#             # save
#             assert sample not in results, 'repeated sample names. can not have in python dictionary'
#             results[sample]=normalized_gexp
#             normalized_matrix = pd.DataFrame.from_dict(results)
#             normalized_matrix.index = DATA.index
#     return normalized_matrix


def get_euclidean(metric, v1, v2):
    '''
    example: ('single', 1, 6) indicates single euclidean distance between value 1 and value 2    
    example: ('mean', list(1,3,4,5), list(6,7,8,9)) indicates euclidean distance between each pair of values (order matters)
    and then the mean euclidean distance of those computed
    '''
    import numpy as np
    import statistics
    # calcuate the eucl distance for pair of values
    if metric == 'single':
        euc = np.linalg.norm(v1 - v2)
        return round(euc, 2)
    
    # calcuate the mean eucl distance for the pair of lists
    if metric == 'mean':
        assert len(v1) == len(v2), 'two lists must be same length'
        euc_list = []
        for i in range(0, len(v1)):
            euc = np.linalg.norm(v1[i] - v2[i])
            euc_list.append(euc)
        mean_euc = statistics.mean(euc_list) 
        return round(mean_euc, 2)

In [ ]:
# Get important feature lists (important genes = TMP ft selected sets)
with open('../data/distance_metric/src/cancer2fts.json', 'r') as fh:
    for line in fh:
        cancer2fts = json.loads(line)

In [ ]:
# read in hcmi data
model_df = df_read(cancer, 'Model')
tumor_df = df_read(cancer, 'Tumor')

# ft reduction (using TMP genes) and filter for shared genes (ex. tumor, model, TMP impt genes) 
genes_keep = intersection( list(tumor_df.columns), list(model_df.columns) )
combined_top_fts = cancer2fts[cancer] # TMP important fts
genes_keep = intersection( genes_keep, combined_top_fts)
model_df = model_df[genes_keep]
tumor_df = tumor_df[genes_keep]

In [ ]:
# # normalize data
# model_df= normalize_data(model_df, 'col')
# tumor_df= normalize_data(tumor_df, 'col')

In [ ]:
print(model_df.shape)
print(tumor_df.shape)

In [ ]:
# now save these files
model_df.to_csv('../data/distance_metric/{}_GEXP/{}_topgenes_Model.tsv'.format(cancer, cancer), sep='\t', index=True)
tumor_df.to_csv('../data/distance_metric/{}_GEXP/{}_topgenes_Tumor.tsv'.format(cancer,cancer), sep='\t', index=True)

# calculate distance (euclidean)


In [ ]:
### Distance between a given tumor and the rest of it's tumor cohort ###
### ex. What is the distance between PAAD_tumor_1 and the rest of the PAAD_tumor cohort? ###
# Calculate the overall mean euc_dist sample 1 is to all other samples (find mean dist in table)
results_sample = []
results_distance = []
results_type = []
results_sample_info = []
for sample_1 in tumor_df.index:
    samples = list(tumor_df.index)
    samples.remove(sample_1) # ensure not comparing the same sample

    # Compute mean sample distance of ONE sample to each other sample (pairwise)
    list_sample_1 =[]
    list_sample_2 =[]
    euclidean_list =[]
    for sample_2 in samples:
        # get expression values across all genes
        value_1 = list(tumor_df.loc[sample_1])
        value_2 = list(tumor_df.loc[sample_2])
        genes = list(tumor_df.columns)

        # calculate mean euc distance between 2 samples (mean across all genes)
        euc_mean = get_euclidean('mean', value_1, value_2)

        # prep for results matrix
        list_sample_1.append(sample_1)
        list_sample_2.append(sample_2)
        euclidean_list.append(euc_mean)

    # View results
    # pd.DataFrame(list(zip(list_sample_1, list_sample_2, euclidean_list)), columns = ['sample1', 'sample2', 'mean_euclidean_dist'])

    # Calculate the overall mean euc_dist sample 1 is to all other samples (find mean dist in table)
    mean_euc_1vAll_samples = round(statistics.mean(euclidean_list), 2)

    # Save
    results_sample.append(sample_1)
    results_distance.append(mean_euc_1vAll_samples)
    results_type.append('mean_euc_1vsALL_samples')
    results_sample_info.append('tumorVStumors_sameTissue')
results_df = pd.DataFrame(list(zip(results_sample, results_distance, results_type, results_sample_info)), 
             columns = ['sample_1', 'distance', 'distance_type', 'sample_info'])
# calculate z-score (within tumors )
darray = np.array(results_df['distance'])
z = stats.zscore(darray)
results_df['z-score_to_tumors']= z

In [ ]:
### Distance between a given tumor and it's derived model ###
### ex. What is the distance between PAAD_tumor_1 and the PAAD_model_1 ? ###

# Create look up matches of model and cancer model
## New FINAL
file = '../src/distance_metric/HCMI_AWG_Model-Tumor-Normal_Linkage_v2.0_2.20.2024.txt'
# file = '../src/distance_metric/HCMI_AWG_Model-Tumor-Normal_Linkage_v2.0_10.27.2023.txt'
ref = pd.read_csv(file, sep='\t')
# ref.index = list(ref['Matched Tumor Aliquot'])

# note some tumors match to multiple models
tumor2model = ref[['Matched Tumor Aliquot', 'Cancer Model Aliquot']]
tumor2model = tumor2model.dropna() #only keep tum:model pairs that have no nan

In [ ]:
# tweak based on mushriqs missing file 
# add any pairs that are missing
mdf = pd.read_csv('../data/distance_metric/src/missing_ohsu_euclidean_distance_all.02.12.2024.csv', sep=',')
# mdf = pd.read_csv('../data/distance_metric/src/missing_ohsu_single_sheet.csv', sep=',')
longform = {
    'BLCA':['Bladder Cancer'],
    'BRCA':['Breast Cancer','Intraductal Carcinoma'],
    'COADREAD': ['Colorectal Cancer', 'Rare Cancer','Serrated Adenoma','Tubular Adenoma','Tubulovilluos Adenoma'],
    'ESO': ['Esophageal Cancer','Stomach Cancer'],
    'HNSC':['Head and Neck'], # ?? need to check if can find this wording in analysis tracker
    'KIRCKICH':['Kidney Cancer'],
    'LGGGBM':['Glioblastoma'],
    'LIHCCHOL':['Extrahepatic Cholangiocarcinoma','Ampulla of Vater','Hepatocellular carcinoma','Intrahepatic Cholangiocarcinoma'],
    'LUNG':['Lung Cancer'],
    'OV':['Ovarian Cancer'],
    'PAAD': ['Pancreatic Cancer'],
    'SARC': ['Bone Cancer','Clear Cell Sarcoma','Desmoid Tumors','Epithelial Sarcoma','Ewing Sarcoma','Intimal Sarcoma','Leiomyosarcoma','Rhabdomyosarcoma','Spindle Cell Sarcoma','Undifferentiated Pleomorphic Sarcoma'],
    'SKCM':['Melanoma'], 
    'UCEC':['Endometrial Cancer'],
}
mdf= mdf[mdf['cancer_type'].isin(longform[cancer])]
mdf = mdf[['matched_tumor_aliquot', 'aliquot_id3', ]].reset_index(drop=True)
mdf.columns = ['Matched Tumor Aliquot','Cancer Model Aliquot']

In [ ]:
# Build on "matched tumor aliquot" column from Missing values file
new = {'Matched Tumor Aliquot':[], 'Cancer Model Aliquot':[]}
for i in range(0, mdf.shape[0]):
    mdf_tumor = mdf['Matched Tumor Aliquot'][i]
    mdf_model = mdf['Cancer Model Aliquot'][i]
    # if tumor not in tracker, then add it
    if mdf_tumor not in list(tumor2model['Matched Tumor Aliquot']):
        #print('adding a completely new tumor (and the model pair)')
        new['Matched Tumor Aliquot'].append(mdf_tumor)
        new['Cancer Model Aliquot'].append(mdf_model)
    # if tumor has a different model match, then add it
    elif list(tumor2model[tumor2model['Matched Tumor Aliquot']==mdf_tumor]['Cancer Model Aliquot'])[0] != mdf_model:
        #print('adding a new tumor-model pair even tho tumor already in list')
        new['Matched Tumor Aliquot'].append(mdf_tumor)
        new['Cancer Model Aliquot'].append(mdf_model)
new_df = pd.DataFrame.from_dict(new)
tumor2model = pd.concat([tumor2model, new_df])
tumor2model = tumor2model.reset_index(drop=True)

In [ ]:
# Build on "matched tumor aliquot" column from Missing values file
new = {'Matched Tumor Aliquot':[], 'Cancer Model Aliquot':[]}
for i in range(0, mdf.shape[0]):
    mdf_tumor = mdf['Matched Tumor Aliquot'][i]
    mdf_model = mdf['Cancer Model Aliquot'][i]
    # if tumor not in tracker, then add it
    if mdf_model not in list(tumor2model['Cancer Model Aliquot']):
        new['Matched Tumor Aliquot'].append(mdf_tumor)
        new['Cancer Model Aliquot'].append(mdf_model)
        print('adding new model {}'.format(mdf_model))
    # if tumor has a different model match, then add it
    elif list(tumor2model[tumor2model['Cancer Model Aliquot']==mdf_model]['Matched Tumor Aliquot'])[0] != mdf_tumor:
        new['Matched Tumor Aliquot'].append(mdf_tumor)
        new['Cancer Model Aliquot'].append(mdf_model)
        print('model {} will now have tumor match {} instead of {}'.format(mdf_model, mdf_tumor,list(tumor2model[tumor2model['Cancer Model Aliquot']==mdf_model]['Matched Tumor Aliquot'])[0] ))
new_df = pd.DataFrame.from_dict(new)
tumor2model = pd.concat([tumor2model, new_df])
tumor2model = tumor2model.reset_index(drop=True)

In [ ]:
# manually update as per v2 Mushriq missing file (4 total samples)
manual_matches = {
    'LGGGBM': [
        {'tumor': 'HCM-BROD-0457-C71-01B','model':'HCM-BROD-0457-C71-85A'},
        {'tumor': 'HCM-BROD-0796-C71-01B', 'model':'HCM-BROD-0796-C71-85A'},
        {'tumor': 'HCM-BROD-0457-C71-01A', 'model':'HCM-BROD-0457-C71-85A'},
        {'tumor': 'HCM-BROD-0796-C71-01B', 'model':'HCM-BROD-0796-C71-85A'},
    ],
    'COADREAD':[
        {'tumor':'HCM-WCMC-1344-C18-01A' ,'model':'HCM-WCMC-1344-C18-85A'},
        {'tumor': 'HCM-CSHL-0603-C18-06A', 'model':'HCM-CSHL-0603-C18-85M'},
        {'tumor': 'HCM-CSHL-0603-C18-06A', 'model':'HCM-CSHL-0603-C18-85A'},
        {'tumor':'HCM-CSHL-0603-C18-06A','model':'HCM-CSHL-0603-C18-85A'},
        {'tumor':'HCM-CSHL-1269-C18-06A','model':'HCM-CSHL-1269-C18-85M'},
        # added feb 2024
        {'tumor': 'HCM-CSHL-0064-C18-01A', 'model': 'HCM-CSHL-0064-C18-86A'},
        {'tumor': 'HCM-CSHL-0161-C18-01A', 'model': 'HCM-CSHL-0161-C18-85A'},
        {'tumor': 'HCM-CSHL-0162-C19-01A', 'model': 'HCM-CSHL-0162-C19-85A'},
        {'tumor': 'HCM-CSHL-0246-C19-01B', 'model': 'HCM-CSHL-0246-C19-86A'},
        {'tumor': 'HCM-CSHL-0603-C18-06A', 'model': 'HCM-CSHL-0603-C18-85M'},
        {'tumor': 'HCM-SANG-0270-C20-01A', 'model': 'HCM-SANG-0270-C20-86A'}
    ],
    'PAAD': [
        {'tumor':'HCM-CSHL-1264-C25-01A', 'model':'HCM-CSHL-1264-C25-85A'},        
    ],
    'SKCM': [
        {'tumor':'HCM-BROD-0444-C43-06A','model':'HCM-BROD-0444-C43-85M'},
        {'tumor':'HCM-BROD-0683-C43-06A','model':'HCM-BROD-0683-C43-85A'},
        {'tumor':'HCM-BROD-0715-C43-06A','model':'HCM-BROD-0715-C43-85B'},
        {'tumor':'HCM-BROD-1119-C43-06A','model':'HCM-BROD-1119-C43-85B'}
    ],
    'ESO': [
        {'tumor':'NA', 'model':'NA'},        
    ],
    'BRCA': [
        {'tumor':'HCM-CSHL-0153-C50-01B', 'model':'HCM-CSHL-0153-C50-85E'},        
    ],
    'BLCA': [
        {'tumor': 'HCM-CSHL-0802-C67-01A', 'model': 'HCM-CSHL-0802-C67-85B'},
        {'tumor': 'HCM-WCMC-0675-C67-02A', 'model': 'HCM-WCMC-0675-C67-85R'},
        {'tumor': 'HCM-WCMC-0948-C67-01A', 'model': 'HCM-WCMC-0948-C67-85A'},
        {'tumor': 'HCM-WCMC-0949-C67-01A', 'model': 'HCM-WCMC-0949-C67-85A'},
        {'tumor': 'HCM-WCMC-0950-C67-01A', 'model': 'HCM-WCMC-0950-C67-85A'},
        {'tumor': 'HCM-WCMC-0951-C67-01A', 'model': 'HCM-WCMC-0951-C67-85A'},
        {'tumor': 'HCM-WCMC-1284-C67-01B', 'model': 'HCM-WCMC-1284-C67-85A'},
        {'tumor': 'HCM-WCMC-1287-C67-01A', 'model': 'HCM-WCMC-1287-C67-85A'}        
    ],
    'OV': [
        {'tumor':'NA', 'model':'NA'},        
    ],
    'LUNG': [
        {'tumor':'HCM-CSHL-0058-C34-01B', 'model':'HCM-CSHL-0058-C34-86A'},        
    ],
    'KIRCKICH': [
        {'tumor':'NA', 'model':'HCM-BROD-0353-C64-85A'},        
        {'tumor':'NA', 'model':'HCM-BROD-0682-C64-85A'},        
    ],
    'SARC': [
        {'tumor':'HCM-BROD-0007-C49-06A','model':'HCM-BROD-0007-C49-85B'},
        {'tumor':'HCM-BROD-0035-C49-06A','model':'HCM-BROD-0035-C49-85M'},
        {'tumor':'HCM-BROD-0036-C41-06A','model':'HCM-BROD-0036-C41-85A'},
        {'tumor':'HCM-BROD-0052-C49-01A','model':'HCM-BROD-0052-C49-85A'},
        {'tumor':'HCM-BROD-0053-C49-06A','model':'HCM-BROD-0053-C49-85M'},
        {'tumor':'HCM-BROD-0234-C49-06A','model':'HCM-BROD-0234-C49-85M'},
        {'tumor':'HCM-BROD-0351-C49-06B','model':'HCM-BROD-0351-C49-85M'},
        {'tumor':'HCM-BROD-0649-C49-06A','model':'HCM-BROD-0649-C49-85B'},
        {'tumor':'HCM-BROD-0762-C49-01A','model':'HCM-BROD-0762-C49-85B'},
        {'tumor':'HCM-BROD-0871-C49-06B','model':'HCM-BROD-0871-C49-85M'},
        {'tumor':'HCM-BROD-1078-C49-06B','model':'HCM-BROD-1078-C49-85B'},
        {'tumor':'HCM-STAN-0840-C49-06A','model':'HCM-STAN-0840-C49-85A'},
     ],
    'UCEC': [
        {'tumor':'NA', 'model':'NA'},        
    ],
    'LIHCCHOL': [
        {'tumor':'HCM-CSHL-0174-C22-01A', 'model':'HCM-CSHL-0174-C22-85A'},        
        {'tumor':'HCM-CSHL-0174-C22-01A', 'model':'HCM-CSHL-0174-C22-86A'},        
    ],
}

         
        
# Add this if not already there or if already there then update the pairings
for ipair in manual_matches[cancer]:
    sample_tumor = ipair['tumor']
    sample_model = ipair['model']
    if sample_tumor not in list(tumor2model['Matched Tumor Aliquot']):
        tumor2model.loc[len(tumor2model.index)] = [sample_tumor, sample_model]
        
    if sample_model not in list(tumor2model['Cancer Model Aliquot']):
        tumor2model.loc[len(tumor2model.index)] = [sample_tumor, sample_model] 
    
    # todo update so handles for if the sample is there but the pairing is off

In [ ]:
list_sample_1 =[]
list_sample_2 =[]
euclidean_list =[]

for sample in results_df['sample_1']: # results_df is of HCMI tumors
    model_from_tumor2model = list(tumor2model[tumor2model['Matched Tumor Aliquot']==sample]['Cancer Model Aliquot'])
    # loop thru each pair (if tumor matches to multiple models)
    for i_model in model_from_tumor2model:
        if i_model in list(model_df.index):
            # get expression values across all genes
            value_1 = list(tumor_df.loc[sample, ])
            value_2 = list(model_df.loc[i_model ,])
            genes = list(tumor_df.columns)
            
            # calculate mean euc distance between 2 samples (mean across all genes)
            euc_mean = get_euclidean('mean', value_1, value_2)

            # prep for results matrix
            list_sample_1.append(sample)
            list_sample_2.append(i_model)
            euclidean_list.append(euc_mean)

            ## View results
            #pd.DataFrame(list(zip(list_sample_1, list_sample_2, euclidean_list)), columns = ['sample1', 'sample2', 'mean_euclidean_dist'])


results_df_2 = pd.DataFrame(list(zip(list_sample_1, list_sample_2, euclidean_list, ['mean_euc']*len(list_sample_1), ['tumorVSmodel_sameTissue']*len(list_sample_1) )), 
             columns = ['sample_1', 'sample_2', 'distance', 'distance_type', 'sample_info'])

# calculate z-score (within tumors )
darray = np.array(results_df_2['distance'])
z = stats.zscore(darray)
results_df_2['z-score_to_model']= z

In [ ]:
# merge 2 df to one easy to ready matrix
# drops tumor row if not found in both df
a = results_df[['sample_1','distance','sample_info', 'z-score_to_tumors']]
a.columns = ['tumor', 'dist_1', 'dist_1_info','z-score_to_tumors']
b = results_df_2[['sample_1', 'sample_2', 'distance', 'sample_info', 'z-score_to_model']]
b.columns = ['tumor', 'model', 'dist_2', 'dist_2_info','z-score_to_model']
final_results = pd.merge(a,b, on='tumor', how='inner')

# now compare dist of model to other_tumors_in_cohort
# <1 model is more similar to tumor (than tumor to tumor)
# 1 tumor and model are the same
# >1 model is less similar to tumor (than tumor to tumor)
final_results['ratio'] = final_results['dist_2']/final_results['dist_1']
final_results = final_results[['tumor', 'model', 'ratio', 'dist_1', 'dist_1_info', 'dist_2', 'dist_2_info','z-score_to_tumors','z-score_to_model']]

In [ ]:
# determine which are similar based on z-score
z_tumors_list = list(final_results['z-score_to_tumors'])
z_models_list = list(final_results['z-score_to_model'])
similar = []
for i in range(0, len(z_tumors_list)):
    z_tum = z_tumors_list[i]
    z_mod = z_models_list[i]
    #print(final_results['tumor'][i],('tumor', 'model'), (z_tum, z_mod))
    if abs(z_tum) <= 3:
        if abs(z_mod) <= 3:
            similar.append('similarTumor:similarModel')
        else:
            similar.append('similarTumor:dissimilarModel')
    else:
        if abs(z_mod) <= 3:
            similar.append('dissimilarTumor:similarModel')
        else:
            similar.append('dissimilarTumor:dissimilarModel')
            
final_results['similar']= similar


In [ ]:
# # sanity check. 
# print('HCM-CSHL-0058-C34-01B' in list(final_results['model']))
# print('HCM-CSHL-0058-C34-01B' in list(final_results['tumor']))

In [ ]:
print(final_results.shape)

In [ ]:
final_results

In [ ]:
final_results.to_csv('../data/distance_metric/main_results/euc_ratio_HCMITumor.Model_{}.tsv'.format(cancer), sep='\t', index=False)

In [ ]:
from collections import Counter
print(Counter(final_results['similar']))

# look at problematic samples
problem_labels = ['similarTumor:dissimilarModel','dissimilarTumor:similarModel', 'dissimilarTumor:dissimilarModel']
issues =final_results[final_results['similar'].isin(problem_labels)].reset_index(drop=True)
issues.to_csv('../data/distance_metric/main_results/outlier_samples_HCMITumor.Model_{}.tsv'.format(cancer), sep='\t',index=False)
issues

In [ ]:
print('\t', cancer)

outliers_zscore = dict()
a = issues[['tumor', 'model', 'similar', 'z-score_to_tumors', 'z-score_to_model']]
outliers_zscore[cancer]=a

outliers_zscore[cancer]

print('TODO update so this is saved')